## Solver development
This is a notebook for showcasing the development of Solvers. As an example a plasma oscillation is simulated with an electrostatic solver and an electromagnetic spectral solver with boris pusher. 

## Set up simulation for plasma oscillation 

In [ ]:
import pipic
from pipic import consts, types
import numpy as np
from numba import cfunc, carray
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display, clear_output, Image

In [ ]:
# simulation variables 
temperature = 1e-6 * consts.electron_mass * consts.light_velocity**2
density = 1e18
debye_length = np.sqrt(temperature / (4 * np.pi * density * consts.electron_charge**2))
plasma_period = np.sqrt(np.pi * consts.electron_mass / (density * consts.electron_charge**2))
l = 128 * debye_length
xmin, xmax = -l / 2, l / 2
field_amplitude = 0.01 * 4 * np.pi * (xmax - xmin) * consts.electron_charge * density
nx = 64
timestep = plasma_period / 128

In [ ]:
# Define functions for initiating the simulation
@cfunc(types.add_particles_callback)
def density_profile(r, data_double, data_int):
    return density

@cfunc(types.field_loop_callback)
def initial_field(ind, r, E, B, data_double, data_int):
    E[0] = field_amplitude * np.sin(4*np.pi * r[0]/ (xmax-xmin))

In [ ]:
# initialize simulation 
sim=pipic.init(solver='electrostatic_1d', # using ecnergy-conserving (ec) solver
               xmin=xmin,xmax=xmax,
               nx=nx)


# add particles according to density_profile
sim.add_particles(name='electron',
                  number= nx*100, # total number of particles to add 
                  density=density_profile.address,
                  charge=consts.electron_charge,
                  mass=consts.electron_mass,
                  temperature=temperature,)

# add field according to initial_field
sim.field_loop(handler=initial_field.address) # setting initial field





##### Define simulation without spectral em-solver and boris pusher solver

In [ ]:
sim_em_solver=pipic.init(solver='fourier_boris', # using fourier_boris solver
               xmin=xmin,xmax=xmax,
               nx=nx)

# add field according to initial_field
sim_em_solver.field_loop(handler=initial_field.address) # setting initial field

# add particles according to density_profile
sim_em_solver.add_particles(name='electron',
                  number= nx*100, # total number of particles to add 
                  density=density_profile.address,
                  charge=consts.electron_charge,
                  mass=consts.electron_mass,
                  temperature=temperature,)


# add neutralizing background
sim_em_solver.fourier_solver_settings(divergence_cleaning=1)
sim_em_solver.advance(0)
sim_em_solver.fourier_solver_settings(divergence_cleaning=0)

In [ ]:
# define functions and arrays for reading and saving field and particle phase space 
field_dd = np.zeros((nx,), dtype=np.double)  # array for saving Ez-field
@cfunc(types.field_loop_callback)
def field_callback(ind, r, E, B, data_double, data_int):
    # read Ez in the xz plane at y=0
    data = carray(data_double, field_dd.shape, dtype=np.double)
    data[ind[0]] = E[0]

particle_dd = np.zeros((64, nx), dtype=np.double)  # array for saving particle (integrated) phase-space

tt = np.zeros((2,), dtype=np.double)  # array for saving particle energy
@cfunc(types.particle_loop_callback)
def particle_callback_momentum(r, p, w, id, data_double, data_int):
    # save particle momentum and position
    data = carray(data_double, tt.shape, dtype=np.double)
    te = np.sqrt(consts.electron_mass**2 * consts.light_velocity**4 + p[0]**2 * consts.light_velocity**2) * w[0]
    tm = p[0] * w[0]
    data[0] += te
    data[1] += tm

tte = np.zeros((2,), dtype=np.double)  # array for saving total field energy
@cfunc(types.field_loop_callback)
def field_callback_energy(ind, r, E, B, data_double, data_int):
    # save total field energy
    data = carray(data_double, tte.shape, dtype=np.double)
    data[0] += 0.5 * (E[0]**2) * (xmax - xmin) / nx
    data[1] += (E[1]*B[2]-E[2]*B[1]) * (xmax - xmin) / nx


pmin = -np.sqrt(consts.electron_mass * temperature)*10 # minimum momentum
pmax = np.sqrt(consts.electron_mass * temperature)*10 # macoutximum momentum
dp = (pmax - pmin) / particle_dd.shape[0] # momentum step
dx = (xmax - xmin) / particle_dd.shape[1] # position step
@cfunc(types.particle_loop_callback)
def particle_callback(r, p, w, id, data_double, data_int):
    data = carray(data_double, particle_dd.shape, dtype=np.double)
    ip = int(particle_dd.shape[0] * (p[0] - pmin) / (pmax - pmin))
    ix = int(particle_dd.shape[1] * (r[0] - xmin) / (xmax - xmin))
    if ip >= 0 and ip < particle_dd.shape[0] and ix < particle_dd.shape[1] and ix >= 0:
        data[ip, ix] += w[0] / (dx * dp) / density #/ (3*density/pmax) / (xmax - xmin) / (ymax - ymin)  # normalize by dz, dp and density
        # save total energy and momentum


#### Create figures and run simulation

In [ ]:
# initialize plot
fig, ax = plt.subplots(2, 2, constrained_layout=True)

# with ES-solver
x_axis = np.linspace(xmin, xmax, nx)
Ez_plot = ax[1,0].plot(x_axis,field_dd[:])[0]
ax[1,0].set_ylim(-field_amplitude, field_amplitude)
zpz_plot = ax[0,0].imshow(particle_dd / (3/pmax),  #!!!!!
             extent=[xmin, xmax,pmin, pmax], 
             aspect='auto', origin='lower', 
             cmap='YlOrBr',vmin=0, vmax=1,
             interpolation = 'none')

# with EM-solver
Ez_plot_we = ax[1,1].plot(x_axis[:],field_dd[:])[0]
ax[1,1].set_ylim(-field_amplitude, field_amplitude)
zpz_plot_we = ax[0,1].imshow(particle_dd / (3/pmax), 
             extent=[xmin, xmax,pmin, pmax], 
             aspect='auto', origin='lower', 
             cmap='YlOrBr',vmin=0, vmax=1,
             interpolation = 'none')

# set titles
ax[0,0].set_title('Electrostatic solver')
ax[0,1].set_title('Spectral solver with  boris pusher')
ax[1,0].set_xlabel('x (cm)')
ax[1,1].set_xlabel('x (cm)')
ax[0,0].set_ylabel('$p_x$ (cm g/s)')
ax[1,0].set_ylabel('$E_x$ (StatV/cm)')

In [ ]:
# ===============================SIMULATION======================================
simulation_steps = int(8 * plasma_period / timestep)
frames = simulation_steps // 8 # number of frames to show in the animation
counter = 0

te_es = []
tm_es = []
te_em = []
tm_em = []
tef_es = []
tmf_es = []
tef_em = []
tmf_em = []

def animate(i):
    sim.advance(time_step=timestep, number_of_iterations=8,use_omp=True)
    sim_em_solver.advance(time_step=timestep, number_of_iterations=8,use_omp=True)

    
    sim.field_loop(handler=field_callback.address, 
                   data_double=pipic.addressof(field_dd),
                   use_omp=True)
    tt.fill(0)
    tte.fill(0)
    particle_dd.fill(0)
    sim.particle_loop(name='electron', 
                      handler=particle_callback.address, 
                      data_double=pipic.addressof(particle_dd))
    sim.particle_loop(name='electron',
                        handler=particle_callback_momentum.address,
                        data_double=pipic.addressof(tt))
    sim.field_loop(handler=field_callback_energy.address,
                     data_double=pipic.addressof(tte))
    tef_es.append(tte[0])
    tmf_es.append(tte[1])
    te_es.append(tt[0])
    tm_es.append(tt[1])
    Ez_plot.set_ydata(field_dd)
    zpz_plot.set_data(particle_dd / (5/pmax))

    
    sim_em_solver.field_loop(handler=field_callback.address, 
                               data_double=pipic.addressof(field_dd),
                               use_omp=True)
    tt.fill(0)
    tte.fill(0)
    particle_dd.fill(0)
    sim_em_solver.particle_loop(name='electron', 
                                  handler=particle_callback.address, 
                                  data_double=pipic.addressof(particle_dd))
    sim_em_solver.particle_loop(name='electron', 
                                  handler=particle_callback_momentum.address,
                                  data_double=pipic.addressof(tt))
    sim_em_solver.field_loop(handler=field_callback_energy.address,
                                 data_double=pipic.addressof(tte))
    tef_em.append(tte[0])
    tmf_em.append(tte[1])
    te_em.append(tt[0])
    tm_em.append(tt[1])
    Ez_plot_we.set_ydata(field_dd)
    zpz_plot_we.set_data(particle_dd / (5/pmax))

    global counter
    clear_output()
    if counter <= frames:
        display(HTML('<pre> Progress: ' + "{:.2f}".format(100*counter/frames) + '</pre>'), display_id = True)
    counter += 1
    return 
    
ani = animation.FuncAnimation(fig, animate, frames=frames, interval = 40)


html = HTML(ani.to_jshtml())
display(html)
plt.close()
